In [1]:
"""
Post-processing of PLANT_POWER.prn from IDA ICE energy simulations
The respective data titles in PLANT_POWER.prn are:
    AHU heating coil power, W
    Water based heating power to zones, W
    AHU cooling coil power, W
    Water based cooling power to zones, W
    Ideal coolers and other local units, W
    Ideal heaters and other local units, W
    Domestic hot water use, W
"""
import os
import numpy as np
import pandas as pd
import tkinter as tk
import matplotlib.pyplot as plt
import matplotlib
from scipy import integrate
from adjustText import adjust_text
from ladybug.epw import EPW

%matplotlib qt 

############################################################
# Plot Specifications
############################################################
plt.style.use('seaborn-v0_8-dark')
font = {'size'   : 10}

matplotlib.rc('font', **font)

############################################################
# Project Information
############################################################
result_folder = "/all_results"
area_folder = "/all_zones"
zone_data_format = ".xlsx"
keyword_search = "Floor area, m2"
############################################################

############################################################
# Define Functions
############################################################
def get_all_file_names_in_directory(directory_path):
    file_names = []
    for dirpath, dirnames, filenames in os.walk(directory_path):
        file_names.extend(file for file in filenames)
    return file_names

### Class definition
class Sim_Result:
    def __init__(self, name, heating, cooling, peak_heat, peak_cool, dyn_heat, stat_heat, dyn_cool, stat_cool, spec_dyn_heat=0, spec_stat_heat=0, spec_dyn_cool=0, spec_stat_cool=0, area=0, spec_heat=0, spec_cool=0):
        self.name = name
        self.heating = heating
        self.cooling = cooling
        self.peak_heat = peak_heat
        self.peak_cool = peak_cool
        self.dyn_heat= dyn_heat
        self.stat_heat= stat_heat
        self.dyn_cool= dyn_cool
        self.stat_cool= stat_cool
        self.area = area
        self.spec_heat = spec_heat
        self.spec_cool = spec_cool
############################################################

############################################################
# Plot settings
############################################################

# Plot dimensions
plt.rcParams.update({'figure.figsize': (10, 8)})

# annotation font size hourly plot
ann_fon_size=6

# Monthly plot top margine in %
top_margin = 20
############################################################

############################################################
# Reading & Processing
############################################################
working_dir = os.path.abspath('')
res_path = working_dir+result_folder
area_path = working_dir+area_folder
all_file_names = get_all_file_names_in_directory(res_path)

print("The following files are found:\n")
print(all_file_names)
print("\n")

### Initialise building lists 
building_list = []

os.chdir(res_path)
for file_name in all_file_names:
    # Read the PRN file and create a DataFrame, discarding the first column starting with a hashtag
    headers = pd.read_csv(file_name, delim_whitespace=True, nrows=0).columns[1:]
    plant_power = pd.read_csv(file_name, delim_whitespace=True, header=None, skiprows=1, names=headers)

### Display the first few rows of the DataFrame to verify it was loaded correctly
#     print(plant_power.head())
#     print(headers)

    # # All units in Watt
    time_stamp = plant_power['time']
    AHU_heating = plant_power["boil_q_1"]
    Water_heating = plant_power["boil_q_2"]
    AHU_cooling = plant_power["chil_q1"]
    Water_cooling = plant_power["chil_q2"]
    Ideal_coolers = plant_power["loccool"]
    Ideal_heaters = plant_power["locheat"]
    hourly_heating = AHU_heating+Water_heating+Ideal_heaters
    #take max of hourly_heating /surface
    hourly_cooling = AHU_cooling+Water_cooling+Ideal_coolers
    #same
    heating_sum = integrate.simps(hourly_heating, time_stamp)
    cooling_sum = integrate.simps(hourly_cooling, time_stamp)

    dyn_heating_sum = integrate.simps(AHU_heating, time_stamp)
    stat_heating_sum = integrate.simps(Water_heating+Ideal_heaters, time_stamp)
    dyn_cooling_sum = integrate.simps(AHU_cooling, time_stamp)
    stat_cooling_sum = integrate.simps(Water_cooling+Ideal_coolers, time_stamp)


    # Use energy demand in kWh
    building = Sim_Result(name = file_name.split(".")[0], heating = heating_sum/1000, cooling = cooling_sum/1000, peak_heat=hourly_heating.max(), peak_cool=hourly_cooling.max(), dyn_heat=dyn_heating_sum/1000, stat_heat=stat_heating_sum/1000, dyn_cool=dyn_cooling_sum/1000, stat_cool=stat_cooling_sum/1000)
    building_list.append(building)

### Read zone areas and calculate specific heating & cooling demand    
os.chdir(area_path)
for building in building_list:
    area_file = building.name + zone_data_format
    zone_data = pd.read_excel(area_file, index_col=None, skiprows=1, header=None)  
    column_count = np.where(zone_data == keyword_search)[1][0]
    row_count = np.where(zone_data[:][column_count].isna())[0][0]
    zone_area = zone_data[1:row_count][column_count]
    # total area of all simulated zones
    all_zone_area = zone_area.astype(float).sum()
    building.area = all_zone_area
    building.peak_heat= building.peak_heat/building.area # in W/m2
    building.peak_cool= building.peak_cool/building.area # in W/m2
    building.spec_heat = building.heating/building.area  # in kWh/m2
    building.spec_cool = building.cooling/building.area  # in kWh/m2
    building.spec_dyn_heat= building.dyn_heat/building.area  # in kWh/m2
    building.spec_stat_heat=building.stat_heat/building.area  # in kWh/m2
    building.spec_dyn_cool= building.dyn_cool/building.area  # in kWh/m2
    building.spec_stat_cool=building.stat_cool/building.area  # in kWh/m2

os.chdir(working_dir)
############################################################

############################################################
# Print & Plot -- WIP
############################################################
### Plot settings
heating_rgb = np.divide([246,135,127],255)
cooling_rgb = np.divide([144,178,238],255)

bar_width = 0.4
label_size = 10
############################################################
### Display all building results. Can be exported to file
# for building in building_list:
#     attrs = vars(building)
#     print('\n'.join("%s: %.2f" % item for item in attrs.items()))

for building in building_list:
    print('Name: ' + building.name)
    print('Zone area: %.2f m2' % building.area)
    print('Total heating demand: %.2f kWh' % building.heating)
    print('Total cooling demand: %.2f kWh' % building.cooling)
    print('Peak heating demand: %.2f W/m2' % building.peak_heat)
    print('Peak cooling demand: %.2f W/m2' % building.peak_cool) 
    print('Specific heating demand: %.2f kWh/m2' % building.spec_heat)#this one for 19.4
    print('Specific cooling demand: %.2f kWh/m2\n' % building.spec_cool) #same    

### Initialisation
names_all = []
spec_heat_all = []
spec_cool_all = []
spec_heat_dyn = []
spec_heat_stat = []
spec_cool_dyn = []
spec_cool_stat = []

for building in building_list:
    names_all.append(building.name.split(".")[0])
    spec_heat_all.append(building.spec_heat)
    spec_cool_all.append(building.spec_cool)
    spec_heat_dyn.append(building.spec_dyn_heat)
    spec_heat_stat.append(building.spec_stat_heat)
    spec_cool_dyn.append(building.spec_dyn_cool)
    spec_cool_stat.append(building.spec_stat_cool)
x = np.arange(len(names_all))

### Plot specific heating & cooling demand
fig1, ax1 = plt.subplots()
ax1.bar(x-0.2, spec_heat_dyn, bar_width, bottom = 0, color = heating_rgb, alpha = 0.5, label="dynamische Heizung") 
ax1.bar(x-0.2, spec_heat_stat, bar_width, bottom = spec_heat_dyn, color = heating_rgb, label="statische Heizung") 
ax1.bar(x+0.2, spec_cool_dyn, bar_width, bottom = 0, color = cooling_rgb, alpha = 0.5, label="dynamische Kühlung") 
ax1.bar(x+0.2, spec_cool_stat, bar_width, bottom = spec_cool_dyn, color = cooling_rgb, label="statische Kühlung") 


### Plot settings
ax1.grid(True)
ax1.set_ylabel('kWh/$m^2$')
ax1.set_title('Spezifischer Nutzenergiebedarf\n', loc='center')
ax1.set_xticks(x,names_all)
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), ncols=2)
ax1.set_position([0.1,0.15,0.8,0.7])
ax1.set_ylim(0, max(max(spec_heat_all,spec_cool_all))*1.5)
for rect in ax1.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()    
    # The height of the bar is the data value and can be used as the label
    label_text = f'{height:.1f}'  # f'{height:.2f}' to format decimal values    
    label_x = x + width / 2
    label_y = y + height /2 
    label_y = y + height + 1   
    # only plot labels greater than given width
    if height > 2:
        ax1.text(label_x, label_y, label_text, ha='center', va='center', fontsize=label_size)
    elif 0.1< height <= 2:
        ax1.text(label_x, label_y, label_text, ha='center', va='center', fontsize=label_size)
#############################################################################################
#Read EPW values
epw = EPW("./Koln.epw")
temperatures = epw.dry_bulb_temperature.values
print(f'Temperature range from EPW has a length of {len(temperatures)}')

'''
'''

The following files are found:

['BF01.prn', 'BF02.prn', 'BF03.prn', 'BF04.prn', 'BF05.prn', 'BF06.prn', 'BF07.prn']


Name: BF01
Zone area: 583.32 m2
Total heating demand: 9578.57 kWh
Total cooling demand: 10889.43 kWh
Peak heating demand: 13.74 W/m2
Peak cooling demand: 21.23 W/m2
Specific heating demand: 16.42 kWh/m2
Specific cooling demand: 18.67 kWh/m2

Name: BF02
Zone area: 516.31 m2
Total heating demand: 10608.08 kWh
Total cooling demand: 8187.43 kWh
Peak heating demand: 52.69 W/m2
Peak cooling demand: 90.98 W/m2
Specific heating demand: 20.55 kWh/m2
Specific cooling demand: 15.86 kWh/m2

Name: BF03
Zone area: 555.00 m2
Total heating demand: 12753.62 kWh
Total cooling demand: 9134.65 kWh
Peak heating demand: 60.85 W/m2
Peak cooling demand: 112.84 W/m2
Specific heating demand: 22.98 kWh/m2
Specific cooling demand: 16.46 kWh/m2

Name: BF04
Zone area: 658.17 m2
Total heating demand: 11593.98 kWh
Total cooling demand: 11531.58 kWh
Peak heating demand: 38.76 W/m2
Peak cooling demand:

'\n'

In [2]:
'''
'''

### Initialisation:
baseload_pctl_h, baseload_watt_h = 0, 0
filter_min = 0.1 # only plot values above this Watt
### Definition
# baseload_pctl_h = 98 # baseload percentile of heating (h) loads
baseload_watt_h = 3.5 * 10**3 # alternatively in W

# Monthly results calculation
mth_labels = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
days_in_mons = [31,29,31,30,31,30,31,31,30,31,30,31]
hrs_in_mons = np.multiply(days_in_mons,24)
hrs_in_mons = np.cumsum(hrs_in_mons)
days_cumu = np.cumsum(days_in_mons)
days_cumubefore = np.concatenate((np.zeros(1), days_cumu[0:-1])).astype(int)
hrs_cumubefore = np.concatenate((days_cumubefore*24, [sum(days_in_mons)*24])).astype(int)


def get_month_day(hour:int) -> str:
    """
    Get the month and day of the year from the hour of the year

    Args:
        hour (int): hour of the year

    Returns:
        str: [description]
    """
    x = [hour]*len(mth_labels)
    df = pd.DataFrame({'data': x}) 
    df['month'] = pd.cut([hour]*len(mth_labels), bins=np.insert(hrs_in_mons,0,0), labels=mth_labels)
    df['month_h'] = pd.cut([hour]*len(mth_labels), bins=np.insert(hrs_in_mons,0,0), labels=np.insert(hrs_in_mons[:-1],0,0))
    df['day'] =np.floor(((df['data'].astype(float)-df['month_h'].astype(float))/24)).astype(int)

    return f'am {df["day"][0]}. {df["month"][0]}'

# A popup window to select which building to display results
def select_building(event):
    global the_chosen_one
    the_chosen_one = all_file_names[building_selection.curselection()[0]]
    print(the_chosen_one + ' is selected.')
    root.destroy()
    return the_chosen_one

root = tk.Tk()
label = tk.Label(root, text='Single click to select one file for display:')
label.grid()
        
items = all_file_names
var = tk.StringVar()
var.set(items)      
        
building_selection = tk.Listbox(root, listvariable=var, selectmode='extended')
building_selection.grid()
building_selection.bind('<<ListboxSelect>>', select_building)

root.mainloop() # end of popup

# Read the selected building file
os.chdir(res_path)
plant_power = pd.read_csv(the_chosen_one, delim_whitespace=True, header=None, skiprows=1, names=headers)
# Set building area of selected building
building_index=int(the_chosen_one.split(".")[0][-1])-1
selected_area = building_list[building_index].area

# # All units in Watt
time_stamp = plant_power['time']
AHU_heating = plant_power["boil_q_1"]
Water_heating = plant_power["boil_q_2"]
AHU_cooling = plant_power["chil_q1"]
Water_cooling = plant_power["chil_q2"]
Ideal_coolers = plant_power["loccool"]
Ideal_heaters = plant_power["locheat"]
hourly_heating = AHU_heating+Water_heating+Ideal_heaters
hourly_cooling = AHU_cooling+Water_cooling+Ideal_coolers
## Definition of static and dynamic heating & cooling
hrly_dyn_heating = AHU_heating
hrly_sta_heating = Water_heating+Ideal_heaters
hrly_dyn_cooling = AHU_cooling
hrly_sta_cooling = Water_cooling+Ideal_coolers

hrly_sta_cooling = pd.Series(hrly_sta_cooling)
monthly_heating =pd.Series(hrly_dyn_heating + hrly_sta_heating)
monthly_cooling =pd.Series(hrly_dyn_cooling + hrly_sta_cooling)

# Definition of a function to determine the sign change for annotations
sign_change = lambda x_value, mid_value: -1 if x_value > mid_value else 1



### Plot Monthly curves Dynamic ###

# Dynamische Heizlast
fig2_dyn, ax2_dyn = plt.subplots()

max_dyn_heating = max(hrly_dyn_heating/1000)
max_dyn_heating_time =time_stamp[hrly_dyn_heating.argmax()]
ax2_dyn.plot(time_stamp, hrly_dyn_heating/1000, color = heating_rgb ,label="Dynamische Heizlast")  
ax2_dyn.annotate(f"Max: {round(max_dyn_heating,2)}kW \n" +get_month_day(max_dyn_heating_time) ,fontsize=ann_fon_size ,xy=(max_dyn_heating_time,max_dyn_heating),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="darkred"),
            xytext=(10*sign_change(max_dyn_heating_time,time_stamp.median()), 20), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

# Dynamische Kühllast
max_dyn_cooling = max(hrly_dyn_cooling/1000)
max_dyn_cooling_time = time_stamp[hrly_dyn_cooling.argmax()]
ax2_dyn.plot(time_stamp, hrly_dyn_cooling/1000, color=cooling_rgb, label="Dynamische Kühllast")
ax2_dyn.annotate(f"Max: {round(max_dyn_cooling,2)}kW \n" +get_month_day(max_dyn_cooling_time),fontsize=ann_fon_size ,xy=(max_dyn_cooling_time,max_dyn_cooling),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="darkblue"),
            xytext=(10*sign_change(max_dyn_cooling_time,time_stamp.median()), 20), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

ax2_dyn.set_xticks(np.insert(hrs_in_mons[:-1],0,0))
ax2_dyn.set_xticklabels(mth_labels)
ax2_dyn.set_ylabel('Energiebedarf [kW]') 
ax2_dyn.set_xlabel('Monate') 
ax2_dyn.set_title('Dynamische Monatliche Heiz- und Kühllast für %s\n' % the_chosen_one)   
ax2_dyn.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncols=2)
ax2_dyn.set_ylim(0, max(max_dyn_cooling,max_dyn_heating)*(1+top_margin/100))
ax2_dyn.grid(True)
### Plot Monthly curves Static ###
# Statische Heizlast
fig2_sta, ax2_sta = plt.subplots()

max_sta_heating = max(hrly_sta_heating/1000)
max_sta_heating_time = time_stamp[hrly_sta_heating.argmax()]
ax2_sta.plot(time_stamp, hrly_sta_heating/1000, color = heating_rgb ,label="Statische Heizlast")  
ax2_sta.annotate(f"Max: {round(max_sta_heating,2)}kW \n"  +get_month_day(max_sta_heating_time),fontsize=ann_fon_size ,xy=(max_sta_heating_time,max_sta_heating),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="darkred"),
            xytext=(10*sign_change(max_sta_heating_time,time_stamp.median()), 20), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

# Statische Kühllast
max_sta_cooling = max(hrly_sta_cooling/1000)
max_sta_cooling_time = time_stamp[hrly_sta_cooling.argmax()]
ax2_sta.plot(time_stamp, hrly_sta_cooling/1000, color=cooling_rgb, label="Statische Kühllast")
ax2_sta.annotate(f"Max: {round(max_sta_cooling,2)}kW \n"  +get_month_day(max_sta_cooling_time),fontsize=ann_fon_size ,xy=(max_sta_cooling_time,max_sta_cooling),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="darkblue"),
            xytext=(10*sign_change(max_sta_cooling_time,time_stamp.median()), 20), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

# ax2_dyn_month.set_xticks(np.arange(0, 9000, 2000))
ax2_sta.set_xticks(np.insert(hrs_in_mons[:-1],0,0))
ax2_sta.set_xticklabels(mth_labels)
ax2_sta.set_ylabel('Energiebedarf [kW]') 
ax2_sta.set_xlabel('Monate') 
ax2_sta.set_title('Statische Monatliche Heiz- und Kühllast für %s\n' % the_chosen_one)   
ax2_sta.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncols=2)
ax2_sta.set_ylim(0, max(max_sta_heating,max_sta_cooling)*(1+top_margin/100))
ax2_sta.grid(True)


### Plot Monthly curves Static + Danamic###
# Statische Heizlast
fig2_sta_dyn, ax2_sta_dyn = plt.subplots()

aY_2= ax2_sta_dyn.twinx()

max_sta_dyn_heating = max(monthly_heating/1000)
max_sta_dyn_heating_time =time_stamp[monthly_heating.argmax()]
ax2_sta_dyn.plot(time_stamp, monthly_heating/1000, color = heating_rgb ,label="Heizlast")  
ax2_sta_dyn.annotate(f"Max: {round(max_sta_dyn_heating,2)}kW \n"  +get_month_day(max_sta_dyn_heating_time),fontsize=ann_fon_size ,xy=(max_sta_dyn_heating_time,max_sta_dyn_heating),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="darkred"),
            xytext=(10*sign_change(max_sta_dyn_heating_time,time_stamp.median()), 20), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

# Statische Kühllast
max_sta_dyn_cooling = max(monthly_cooling/1000)
max_sta_dyn_cooling_time = time_stamp[monthly_cooling.argmax()]
ax2_sta_dyn.plot(time_stamp, monthly_cooling/1000, color=cooling_rgb, label="Kühllast")
ax2_sta_dyn.annotate(f"Max: {round(max_sta_dyn_cooling,2)}kW \n"  +get_month_day(max_sta_dyn_cooling_time),fontsize=ann_fon_size ,xy=(max_sta_dyn_cooling_time,max_sta_dyn_cooling),  xycoords='data',
            bbox=dict(boxstyle="round", fc="none", ec="darkblue"),
            xytext=(10*sign_change(max_sta_dyn_cooling_time,time_stamp.median()), 20), textcoords='offset points', ha='center',
            arrowprops=dict(arrowstyle="->"))

aY_2.plot( temperatures, color ='grey', label='Temperatur', alpha =0.5)

# ax2_dyn_month.set_xticks(np.arange(0, 9000, 2000))
ax2_sta_dyn.set_xticks(np.insert(hrs_in_mons[:-1],0,0))
ax2_sta_dyn.set_xticklabels(mth_labels)
ax2_sta_dyn.set_ylabel('Energiebedarf [kW]') 
aY_2.set_ylabel('Temperatur [°C]')
ax2_sta_dyn.set_xlabel('Monate') 
ax2_sta_dyn.set_title('Monatliche Heiz- und Kühllast für %s\n' % the_chosen_one)   
ax2_sta_dyn.legend(loc='upper left', bbox_to_anchor=(0.5, -0.1), ncols=2)
aY_2.legend(loc='upper right', bbox_to_anchor=(0.5, -0.1), ncols=1)
ax2_sta_dyn.set_ylim(0, max(max_sta_dyn_heating,max_sta_dyn_cooling)*(1+top_margin/100))
ax2_sta_dyn.grid(True)


### Jahresdauerlinien/Load Duration Curve###
sorted_heating = -np.sort(-hourly_heating)
sorted_cooling = -np.sort(-hourly_cooling)
filtered_heating = sorted_heating[np.where(sorted_heating>filter_min)[0]]
filtered_cooling = sorted_cooling[np.where(sorted_cooling>filter_min)[0]]

fig3, (ax3, ax4) = plt.subplots(2, 1)
ax3.grid(True)
ax4.grid(True)
ax3.plot(time_stamp, sorted_heating/1000, color = heating_rgb, label="Heizlast")  
ax4.plot(time_stamp, sorted_cooling/1000, color = cooling_rgb, label="Kühllast") 
ax3.set_xlim(0, 9000)
ax4.set_xlim(0, 9000)
ax3.set_title('Jahresdauerlinien für %s\n' % the_chosen_one) 

if baseload_watt_h:
    print('Baseload given in watt')
    baseload_where_h = time_stamp[np.where(
        sorted_heating == min(sorted_heating, key=lambda x:abs(x-baseload_watt_h)))[0][0]]     
elif not baseload_watt_h and baseload_pctl_h:
    print('Baseload given in percentile')
    baseload_watt_h = np.percentile(sorted_heating, baseload_pctl_h)
    baseload_where_h = np.percentile(time_stamp, 100-baseload_pctl_h)    

if baseload_watt_h:
    ax3.axhline(baseload_watt_h/1000, 0, baseload_where_h/time_stamp[len(time_stamp)-1], 
                color='r', linestyle='--', label='Horizontal Line')
    ax3.fill_between(time_stamp, baseload_watt_h/1000, sorted_heating/1000, 
                     where=(baseload_watt_h < sorted_heating), 
                     color=heating_rgb, alpha=1, interpolate=True)
    ax3.fill_between(time_stamp, np.fmin(sorted_heating,baseload_watt_h)/1000, 
                     color=heating_rgb, alpha=0.5,interpolate=True)




# Cut load hours under filter_min (shorten the tail of the plot)
# fig4, (ax5, ax6) = plt.subplots(2, 1)
# ax5.plot(time_stamp[np.where(sorted_heating>filter_min)[0]], 
#          filtered_heating/1000, color = heating_rgb, label="Heizlast")  
# ax6.plot(time_stamp[np.where(sorted_cooling>filter_min)[0]], 
#          filtered_cooling/1000, color = cooling_rgb, label="Kühllast") 
# ax5.set_xlim(0, int(time_stamp[len(np.where(sorted_cooling>filter_min)[0])]))
# ax6.set_xlim(0, int(time_stamp[len(np.where(sorted_cooling>filter_min)[0])]))

##Definition static and dynamic heating & cooling
# Monthly plot: separation of dynamic & static


plt.show()

'''
'''

BF03.prn is selected.
Baseload given in watt


'\n'

In [3]:
## Get Energy Needs (Heating, Cooling) for the chosen building
###a prompt will appear, write down the virtual area factor there for calculating energy needs for the whole building

from tkinter import simpledialog

name=the_chosen_one.split('.')[0]
chosen_building=None
for building in building_list:
    if(building.name==name):
        chosen_building=building

ROOT = tk.Tk()
ROOT.withdraw()
# the input dialog
virtual_area = simpledialog.askstring(title="Test", prompt="Write down the virtual area factor")
virtual_area=float(virtual_area)

#Peak demands
peak_heating= chosen_building.peak_heat/1000000*virtual_area
peak_cooling = chosen_building.peak_cool/1000000*virtual_area
#Specific demands
spec_heating =  chosen_building.spec_heat/1000*virtual_area
spec_cooling =  chosen_building.spec_cool/1000*virtual_area

print('Peak heating demand: %.2f MW' % peak_heating)
print('Peak cooling demand: %.2f MW' % peak_cooling) 
print('Heating demand: %.2f MWh' % spec_heating)
print('Cooling demand: %.2f MWh\n' % spec_cooling) 

#create plots

def percentage_plot_horizontal(values, labels, title):
    
    # Calculate percentages
    total = sum(values)
    percentages = [(value / total) * 100 for value in values]
    
    # Create the bar plot
    fig, ax = plt.subplots()
    bars = ax.bar(labels, percentages, color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0'])
    ax.set_ylabel('Prozentteil [%]') 
    # Add percentage labels above bars
    for bar, percentage in zip(bars, percentages):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height, f"{percentage:.1f}%", ha='center', va='bottom')
    
    # Title for the plot
    plt.title(title)
    
    # Display the plot
    plt.show()

def plot_vertical(values, labels, title, xaxis):
    
    # Calculate the cumulative sum for horizontal stacking
    cumsum = np.cumsum(values)
    left = np.roll(cumsum, shift=1)
    left[0] = 0
    
    # Colors for each segment
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0']
    
    # Specify the height of the bars
    bar_height = 0.5
    
    # Create the horizontal bar plot
    fig, ax = plt.subplots()
    bars = ax.barh(0, values, left=left, height=bar_height, color=colors, edgecolor='white')
    
    # Add value labels inside the bars
    for bar, value in zip(bars, values):
        width = bar.get_width()
        ax.text(bar.get_x() + width / 2, bar.get_y() + bar.get_height() / 2, f"{round(value, 1)}",
                ha='center', va='center', color='black', fontsize=10)
    ax.legend(bars, labels, loc='lower right')
    # Remove y-axis ticks
    ax.yaxis.set_visible(False)
    
    # Adjust the y-limit to fit the bar height
    ax.set_ylim(-bar_height, bar_height)
    ax.set_xlabel(xaxis) 
    # Title for the plot
    plt.title(title)
    
    # Display the plot
    plt.show()
    
percentage_plot_horizontal([spec_heating, spec_cooling], ['Heizung', 'Kühlung'], 'Nutzerenergie')
plot_vertical([spec_heating, spec_cooling], ['Heizung', 'Kühlung'], 'Nutzerenergie', 'Nutzerenergie MWh')

Peak heating demand: 12.17 MW
Peak cooling demand: 22.57 MW
Heating demand: 4595.90 MWh
Cooling demand: 3291.77 MWh



In [7]:
# Get Energy Needs (Heating, Cooling) for all buildings
###a prompt will appear, write down the virtual area factors there for calculating energy needs for the whole buildings

data = [[building.spec_heat/1000, building.spec_cool/1000] for building in building_list]

# Function to create pop-up dialog for input
def get_factors():
    factors = []
    
    # Create a tkinter root window (hidden)
    root = tk.Tk()
    root.withdraw()
    
    # Create a new tkinter Toplevel window
    top = tk.Toplevel(root)
    top.title("Enter Surface Factors")
    
    # Create a list to store entry widgets
    entry_widgets = []
    
    # Function to retrieve factors when OK button is pressed
    def ok_pressed():
        nonlocal factors
        factors = []
        for entry in entry_widgets:
            factor_str = entry.get().strip()
            if factor_str:
                factors.append(float(factor_str))
            else:
                factors = None
                break
        top.destroy()  # Close the dialog window
    
    # Create entry boxes for factors
    for i in range(len(data)):
        # Label for item number
        label = tk.Label(top, text=f"Virtual Surface for {building_list[i].name} in m2:")
        label.grid(row=i, column=0, padx=10, pady=5, sticky=tk.E)
        
        # Entry box for factor input
        entry = tk.Entry(top)
        entry.grid(row=i, column=1, padx=10, pady=5)
        entry_widgets.append(entry)
    
    # OK button to submit factors
    ok_button = tk.Button(top, text="OK", command=ok_pressed)
    ok_button.grid(row=len(data), columnspan=2, pady=10)
    
    # Wait for user to close the window
    top.wait_window()
    
    return factors

factors = get_factors()
if factors is None:
    print("Factor input cancelled.")
else:
    # Multiply each value list by its respective factor
    multiplied_data = [[value * factor for value in values] for values, factor in zip(data, factors)]

    # Colors for each segment
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0']
    
    # Calculate the global x-axis limit
    max_sum = max([sum(values) for values in multiplied_data]) * 1.1  # 10% extra space
    
    # Create stacked bar plots with the same x-axis limits
    fig, ax = plt.subplots(len(data), 1, figsize=(10, 6))
    
    for i, values in enumerate(multiplied_data):
        # Calculate cumulative sums for horizontal stacking
        cumsum = np.cumsum(values)
        left = np.roll(cumsum, shift=1)
        left[0] = 0
        
        # Create horizontal stacked bar plot
        bars = ax[i].barh(0, values, left=left, color=colors[:len(values)], edgecolor='white')
        
        # Add value labels inside the bars
        for bar, value in zip(bars, values):
            width = bar.get_width()
            ax[i].text(bar.get_x() + width / 2, bar.get_y() + bar.get_height() / 2, f"{value:.1f}",
                       ha='center', va='center', color='black', fontsize=10)
        
        # Set y-axis properties
        ax[i].set_ylim(-0.5, 0.5)  # Adjusted for bar height
        
        # Set x-axis limits based on the global maximum sum
        ax[i].set_xlim(0, max_sum)
        ax[i].yaxis.set_visible(False)
        
        # Title for each subplot
        ax[i].set_title(f'Energiebedarf für {building_list[i].name} in MWh')

        
    fig.legend(bars, ['Heizung', 'Kühlung'], loc='lower right', frameon=False)
    
    # Adjust layout for better fit
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15) 
    # Display the plot
    plt.show()

In [5]:
'''
'''
# Monthly results calculation
mth_labels = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
days_in_mons = [31,29,31,30,31,30,31,31,30,31,30,31]
hrs_in_mons = np.multiply(days_in_mons,24)
days_cumu = np.cumsum(days_in_mons)
days_cumubefore = np.concatenate((np.zeros(1), days_cumu[0:-1])).astype(int)
hrs_cumubefore = np.concatenate((days_cumubefore*24, [sum(days_in_mons)*24])).astype(int)

# Calculate Monthly energy demand
heating_monthly = []
cooling_monthly = []
for i in range(len(hrs_cumubefore)-1):
    start_time_stamp = np.argmin(np.abs(time_stamp - hrs_cumubefore[i]))
    end_time_stamp = np.argmin(np.abs(time_stamp - hrs_cumubefore[i+1]))
#     print(time_stamp[start_time_stamp], time_stamp[end_time_stamp])
    heating_thismon = integrate.simps(hourly_heating[start_time_stamp:end_time_stamp], 
                                      time_stamp[start_time_stamp:end_time_stamp])
    cooling_thismon = integrate.simps(hourly_cooling[start_time_stamp:end_time_stamp], 
                                      time_stamp[start_time_stamp:end_time_stamp])
    heating_monthly.append(heating_thismon) # Wh
    cooling_monthly.append(cooling_thismon) # Wh

### Plot monthly heating & cooling demand in kWh/m2
fig5, ax7 = plt.subplots()
ax7.grid(True)
ax7.bar(np.arange(len(hrs_cumubefore)-1)-0.2, np.divide(heating_monthly/selected_area,1000), bar_width, color = heating_rgb, label="Heizung") 
ax7.bar(np.arange(len(hrs_cumubefore)-1)+0.2, np.divide(cooling_monthly/selected_area,1000), bar_width, color = cooling_rgb, label="Kühlung") 
ax7.set_xticks(np.arange(len(hrs_cumubefore)-1),mth_labels)
for rect in ax7.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()    
    # The height of the bar is the data value and can be used as the label
    label_text = f'{height:.1f}'  # f'{height:.2f}' to format decimal values    
    label_x = x + width/2
    label_y = y + height/2   
    # only plot labels greater than given width
    if height > 2:
        ax7.text(label_x, label_y, label_text, ha='center', va='center', fontsize=label_size)
    elif 0.1< height <= 2:
        ax7.text(label_x, label_y, label_text, ha='center', va='center', fontsize=label_size)
ax7.legend(loc='upper center', bbox_to_anchor=(0.5, -0.06), ncols=2)
ax7.set_ylabel('kWh/m2')
ax7.set_title('Monatlicher Spezifischer Nutzenergiebedarf\n', loc='center')   
        
        
# Monthly plot: separation of dynamic & static
hrly_dyn_heating = AHU_heating/selected_area
hrly_sta_heating = (Water_heating+Ideal_heaters)/selected_area
hrly_dyn_cooling = AHU_cooling/selected_area
hrly_sta_cooling = (Water_cooling+Ideal_coolers)/selected_area

# Calculate Monthly energy demand
dyn_heating_monthly = []
sta_heating_monthly = []
dyn_cooling_monthly = []
sta_cooling_monthly = []

for i in range(len(hrs_cumubefore)-1):
    start_time_stamp = np.argmin(np.abs(time_stamp - hrs_cumubefore[i]))
    end_time_stamp = np.argmin(np.abs(time_stamp - hrs_cumubefore[i+1]))
#     print(time_stamp[start_time_stamp], time_stamp[end_time_stamp])
    dyn_heating_thismon = integrate.simps(hrly_dyn_heating[start_time_stamp:end_time_stamp], 
                                      time_stamp[start_time_stamp:end_time_stamp])
    sta_heating_thismon = integrate.simps(hrly_sta_heating[start_time_stamp:end_time_stamp], 
                                      time_stamp[start_time_stamp:end_time_stamp])    
    dyn_cooling_thismon = integrate.simps(hrly_dyn_cooling[start_time_stamp:end_time_stamp], 
                                      time_stamp[start_time_stamp:end_time_stamp])
    sta_cooling_thismon = integrate.simps(hrly_sta_cooling[start_time_stamp:end_time_stamp], 
                                      time_stamp[start_time_stamp:end_time_stamp])    
    dyn_heating_monthly.append(dyn_heating_thismon) # Wh
    sta_heating_monthly.append(sta_heating_thismon) # Wh
    dyn_cooling_monthly.append(dyn_cooling_thismon) # Wh
    sta_cooling_monthly.append(sta_cooling_thismon) # Wh

### Plot monthly heating & cooling demand in kWh/m2
fig6, ax8 = plt.subplots()
ax8.grid(True)
ax8.bar(np.arange(len(hrs_cumubefore)-1)-0.2, np.divide(dyn_heating_monthly,1000), bar_width,
        bottom = 0, color = heating_rgb, alpha = 0.5, label="dynamische Heizung") 
ax8.bar(np.arange(len(hrs_cumubefore)-1)-0.2, np.divide(sta_heating_monthly,1000), bar_width, 
        bottom = np.divide(dyn_heating_monthly,1000), color = heating_rgb, label="statische Heizung") 
ax8.bar(np.arange(len(hrs_cumubefore)-1)+0.2, np.divide(dyn_cooling_monthly,1000), bar_width,
        bottom = 0, color = cooling_rgb, alpha = 0.5, label="dynamische Kühlung") 
ax8.bar(np.arange(len(hrs_cumubefore)-1)+0.2, np.divide(sta_cooling_monthly,1000), bar_width, 
        bottom = np.divide(dyn_cooling_monthly,1000), color = cooling_rgb, label="statische Kühlung") 
ax8.set_xticks(np.arange(len(hrs_cumubefore)-1),mth_labels)
for rect in ax8.patches:
    # Find where everything is located
    height = rect.get_height()
    width = rect.get_width()
    x = rect.get_x()
    y = rect.get_y()    
    # The height of the bar is the data value and can be used as the label
    label_text = f'{height:.1f}'  # f'{height:.2f}' to format decimal values    
    label_x = x + width / 2
    label_y = y + height / 2   
    # only plot labels greater than given width
    if height >0.2:
        ax8.text(label_x, label_y, label_text, ha='center', va='center', fontsize=label_size)
#     elif 0.1< height <= 2:
#         ax8.text(label_x, label_y, label_text, ha='center', va='center', fontsize=label_size)
ax8.legend(loc='upper center', bbox_to_anchor=(0.5, -0.06), ncols=4)
ax8.set_ylabel('kWh/m2')
ax8.set_title('Monatlicher Spezifischer Nutzenergiebedarf\n', loc='center')        

'''
'''

'\n'

In [25]:
# TODOS

# dry bulb temperature ambient - read from weather file, most likely epw. sometimes dwd and overlay on the monthly energy needs - done with epw

# upscaling with GFA & factors - instead of excel - done


# input parameters
# internal gains & schedules


# electricity demand
# standard library of measures

In [26]:
# WIP!!!!
# cumulative area peak & base load 


# heating_sum = integrate.simps(hourly_heating, time_stamp)
# cooling_sum = integrate.simps(hourly_cooling, time_stamp)
# ax3.fill_between(time_stamp, baseload_watt_h/1000, sorted_heating/1000, 
#                  where=(baseload_watt_h < sorted_heating), 
#                  color=heating_rgb, alpha=1, interpolate=True)
# ax3.fill_between(time_stamp, np.fmin(sorted_heating,baseload_watt_h)/1000, 
#                  color=heating_rgb, alpha=0.5,interpolate=True)